# Brain Classification

Here is the PyTorch code for the Brain downstream task

The dataset is internal from the Ulm University Medical Center and can not be released publicy due to strict data securety laws in Germany.

### Preferences:

You need to fill out the first cell with your preferences:

In [18]:
# Folder where the results will be saved to: (Create a folder on your computer and type the filename here)
root_dir = "/home/wolfda/Data/Spark/Downstream/Results"

# Folder were you saved the data  
data_dir = "/home/wolfda/Data/Cathi/Train_Test" 

# Choose names for Weights & Biases
Project = "Brain"  # Name of the WandB Project
Run = "SparK_1" # Name of the Run inside the Project
wandb_tag= ["Tag"] # You can give an Tag to the Run here if you want

# Pre-Training
# Chose if you want to use a pre-trained model (Ture == Pre-Trained weights are loaded || Fales == No pre-training, model is trained from scratch)
preTrain = True 
# Type the filename to the downloaded pre-training Checkpoint here (.ckpt or .pth) [Download from Read.Me page]
pretrained_weights = "/home/wolfda/Data/Spark/PreTrain/BYOL/LIDC-Hash6/epoch=300-train_loss=-3.90.ckpt" 
# Choose the Pre-Training Method: 
pre_train = "BYOL" # You can write: "SwAV" "SparK" "BYOL" "MoCo" (this must match the Checkpoint)

# Choose the model
backbone_model = "ResNet" # We have only implemeted ResNet50 -> "ResNet"

# Train Dataset Reduction: 
# What percentage of the downstream task data should be used for training? 
data_percentage = 1 # 1 for all Data; 0.5 for half the Data; ...
# Seed for Reduction
seed = 42

# Number of Epochs: 
Epochen = 70

# Learning Rate
lr = 1e-4 
WeightDecay = 0.0005 

# Batch Size
bs = 64 

# Freeze Encoder in the beginning (only the linear layer is trained) 
first_frozen = True # True: first Frozen, after n epochs open + smaller learning rate || False: Train entire encoder form Start
unfreeze = 10 # From which epoch should the encoder be open (ATTENTION: If you change the model, the encoder must be called "backbone") 


### Imports:

In [19]:
import os
import PIL
import numpy as np

# Monai
from monai.transforms import (
    AsDiscrete,
    Compose,
    Activations,
    LoadImage,
    RandFlip,
    EnsureType,
    Resize,
)
from monai.config import print_config

# Output Metrics
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, f1_score, accuracy_score, classification_report, ConfusionMatrixDisplay, precision_score

# Weights & Biasis
from pytorch_lightning.loggers import WandbLogger
# Wandb Callbacks from hydra Template 
import wandb_callbacks as wbcall
# Logger
from logging import getLogger
logger = getLogger()

# PyTorch 
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import pytorch_lightning
from pytorch_lightning import callbacks
import torchmetrics

# Random shuffle
import random

# Seed Everything 
pytorch_lightning.seed_everything(42, workers=True)

2023-11-23 13:26:14,101 - Global seed set to 42


42

### Loss + Optimizer

In [20]:
# Loss, Optimizer 
loss = torch.nn.CrossEntropyLoss() # NLLLoss() falls Netz schon softmax macht | CrossEntropyLoss() falls Netz noch keine Softmax macht  || in (): weight=torch.FloatTensor([0.3,0.7])
optim = "Adam" # Optimizer 
# Speichert für wandb:
info={"Path_save": root_dir, "Path_data": data_dir, "seed": seed, "PreTrain_Weights": pretrained_weights, "Epochs": Epochen, "first_frozen": first_frozen, "Epochs Freeze untill": unfreeze, "Learning_Rate": lr,  "Bs": bs, "Optim Weight_Decay": WeightDecay,}

### Dataset:

##### Load Train and Test data:

In [21]:
# Train 
train_dir = data_dir + "/train"

class_names = sorted(x for x in os.listdir(train_dir)
                     if os.path.isdir(os.path.join(train_dir, x)))
num_class = len(class_names)
image_files = [
    [
        os.path.join(train_dir, class_names[i], x)
        for x in os.listdir(os.path.join(train_dir, class_names[i]))
    ]
    for i in range(num_class)
]
num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)
image_width, image_height = PIL.Image.open(image_files_list[0]).size
n_classes = len(class_names)

train_val_x = image_files_list
train_val_y = image_class
print("----------------- Train------------------")
print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label names: {n_classes}")
print(f"Label counts: {num_each}")


# Test 
test_dir = data_dir + "/test"

class_names = sorted(x for x in os.listdir(test_dir)
                     if os.path.isdir(os.path.join(test_dir, x)))
num_class = len(class_names)
image_files = [
    [
        os.path.join(test_dir, class_names[i], x)
        for x in os.listdir(os.path.join(test_dir, class_names[i]))
    ]
    for i in range(num_class)
]
num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)
image_width, image_height = PIL.Image.open(image_files_list[0]).size
n_classes = len(class_names)

test_x = image_files_list
test_y = image_class
print("----------------- Test------------------")
print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label names: {n_classes}")
print(f"Label counts: {num_each}")

----------------- Train------------------
Total image count: 170
Image dimensions: 670 x 630
Label names: ['a_without', 'b_with']
Label names: 2
Label counts: [85, 85]
----------------- Test------------------
Total image count: 29
Image dimensions: 670 x 630
Label names: ['a_without', 'b_with']
Label names: 2
Label counts: [15, 14]


###### Random Train/Val Split out of the Train Dataset

In [22]:
val_frac = 0.15  
length = len(train_val_x)
indices = np.arange(length)
np.random.seed(seed) 
np.random.shuffle(indices)

val_split = int(val_frac * length)
train = (length - val_split)
val_indices = indices[:val_split]
train_indices = indices[val_split:(int(train)+val_split)]

train_x = [train_val_x[i] for i in train_indices]
train_y = [train_val_y[i] for i in train_indices]
val_x = [train_val_x[i] for i in val_indices]
val_y = [train_val_y[i] for i in val_indices]

num_each_train = [len(train_y)-sum(train_y), sum(train_y)]
num_each_val = [len(val_y)-sum(val_y), sum(val_y)]

print("Training count:", len(train_x))
print("Lable counts:", num_each_train)
print("Validation count:" ,len(val_x))
print("Lable counts:" ,num_each_val)

Training count: 145
Lable counts: [71, 74]
Validation count: 25
Lable counts: [14, 11]


###### Train Dataset Reduction

In [23]:
# Seed
np.random.seed(seed)
# Zip the two lists togethe
zipped_lists = list(zip(train_x, train_y))
# Shuffle the zipped list
random.shuffle(zipped_lists)
# Calculate the new length of the lists
new_length = int(len(zipped_lists) * data_percentage)
# Slice the zipped list to get the first half
reduced_zipped_lists = zipped_lists[:new_length]
# Unzip the reduced zipped list into two separate lists
reduced_train_x, reduced_train_y = zip(*reduced_zipped_lists)
train_x = list(reduced_train_x)
train_y = list(reduced_train_y)

print("Reduced Train X", len(train_x))
print("Reduced Train Y", len(train_y))
print("Class 0", train_y.count(0))
print("Class 1", train_y.count(1))

Reduced Train X 145
Reduced Train Y 145
Class 0 71
Class 1 74


### Transforms

In [24]:
# Gray scale Transforms:
# If gray scale: three times the same image together -> to get 3 chaneles (Model is build for RGB (=3 Channel) image)
class gray_to_rgb(object):
    def __init__(self):
        pass
    def __call__(self, image):
        if len(image.shape) == 2: # gray
            image = np.stack((image, image, image), 2)
            return image
        if image.shape[2] == 4: # Falls (x,y,4) -> (x,y,3)
            image = np.stack((image[...,0], image[...,1], image[...,2]), 2)
            return image
        else:
            return image
    

In [25]:
# To uint8 (because PIL needs uint8)                                
class to_uint8(object):
    def __init__(self):
        pass
    def __call__(self, image):
        image = image.astype(np.uint8)
        return image

In [26]:
# maps values form [255,0] -> [1,0]
class scale(object):
    def __init__(self):
        pass
    def __call__(self, image):
        # map values from [from_min, from_max] to [to_min, to_max]
        # image: input array
        from_range = 255 - 0
        to_range = 1 - 0
        scaled = np.array((image - 0) / (float(from_range)+0.00001), dtype=float)
        return 0 + (scaled * to_range)

In [27]:
# Transforms

# Mean and Std of the gray scale values of the training dataset
mean = [0.5600, 0.5596, 0.5596]
std = [0.2905, 0.2907, 0.2908]

train_transforms = transforms.Compose(
    [   
        # Load Image
        LoadImage(image_only=True), # Loads Image (nii, nii.gz -> NibabelReader), (png, jpg, bmp -> PILReader), (npz, npy -> NumpyReader), (nrrd -> NrrdReader), (DICOM file -> ITKReader)
        # Numpy Array [HxWxChannels] [Range: 0-255]
        
        # If Gray scale: (x,y) -> 3x(x,y) 
        gray_to_rgb(),
        
        # To PIL -------------------------------------
        to_uint8(), # PIL needs numpy array with uint8, converts to [0-255, 8 bit]
        transforms.ToPILImage(), # numpy to PIL (Numpy Image needs to be: [HxWxChannels])
        
        # ... PIL Transforms can be added here ...
        
        # OR ----------------------------------------
        # If no PIL before: 
        # Scale [255,0] -> [1,0]
        #scale(),
        # -------------------------------------------
        
        # PIL/Numpy to Torch --------------------------------
        # [HxWxChannels] -> [ChannelsxWxH]
        # If PIL before: [0-225] -> [0-1] || If numpy Numpy: [0-225] -> [0-225]
        transforms.ToTensor(),
        
        # ... Torch or Monai Transforms can be added here ... 
        
        # Normalize:
        # Needs to be: [0,1] sonst klappt es nicht
        transforms.Normalize(mean=mean, std=std), # Normalize (Mean should be arround 0 now)
        
        # Resize:
        Resize(spatial_size=(224,224)), # Resize to [Channelsx224x224]
        
        #RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        
        EnsureType(), # no matter what -> to Torch (not needed here as we already have Torch)
    ]
)

val_transforms = Compose(
    [LoadImage(image_only=True),
     gray_to_rgb(),  
     
     # PILL ----------
     to_uint8(), 
     transforms.ToPILImage(), 
     
     # OR-----
     #scale(),
     
     transforms.ToTensor(), 
     transforms.Normalize(mean=mean, std=std),
     Resize(spatial_size=(224,224)),
     EnsureType(),
    ])

y_pred_trans = Compose([EnsureType(), Activations(softmax=True)])
y_trans = Compose([EnsureType(), AsDiscrete(to_onehot=num_class)])

In [28]:
# PT Dataset

class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]

### Load Model and Pre-Training
Here, the ResNet50 model is loaded and initialized with the pre-training checkpoints:

#### Expected output of this Cell: 
If this appears, everything is correct: 
- missing_keys= ['fc.weight', 'fc.bias'] (beacuse the last fully connected layer was not pre-trained) 
- unexpected_keys= 
    - MoCo: All "encoder_k" layers (because MoCo has 2 encoders and we use only encoder_q)
    - BYOL: All "online_network.projector" and "target_network.encoder" layers (because BYOL has 2 encoders and we only the online_network.encoder)
    - SwAV: All "projection_head" layers (beacuse SwAV has an aditional projection head for the online clustering) 
    - SparK: []
    
    
#### You can insert a new model here:
To check the names of the model and the pre-training checkpoints, use:

    model = models.xyz()
    state_dict = torch.load(pretrained_weights)
    for k, v in model.state_dict().items():
        print(k)
    print("--------------------------------------------------------------------------------")
    for k, v in state_dict.items():
        print(k)

In [29]:
# Model-----------------------------------------------------------------------------

if backbone_model == "ResNet":
    res_model = models.resnet50() # PyTorch Torchvision Resnet50 Model
else:
    print("No Model")
    

# Pre-Training --------------------------------------------------------------------------------
if preTrain == True:
    
    
    # Load pre-training weights
    state_dict = torch.load(pretrained_weights)  #pretrained_weights
    
    # Match the correct name of the layers between pre-trained model and PyTorch ResNet
    # Extraction:
    if "module" in state_dict: # (SparK)
        state_dict = state_dict["module"] 
    if "state_dict" in state_dict: # (SwAV, MoCo, BYOL) 
        state_dict = state_dict["state_dict"]
    # Replacement:
    if pre_train == "SparK" or pre_train == "SwAV":
        state_dict = {k.replace("model.", ""): v for k, v in state_dict.items()} 
    elif pre_train == "MoCo":
        state_dict = {k.replace("encoder_q.", ""): v for k, v in state_dict.items()} 
    elif pre_train == "BYOL":
        state_dict = {k.replace("online_network.encoder.", ""): v for k, v in state_dict.items()} 

    # Initialisation of the ResNet model with pre-training checkpoints
    msg = res_model.load_state_dict(state_dict, strict=False)
    
    
    # Check if it works
    print(format(msg))

_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=['online_network.projector.model.0.weight', 'online_network.projector.model.1.weight', 'online_network.projector.model.1.bias', 'online_network.projector.model.1.running_mean', 'online_network.projector.model.1.running_var', 'online_network.projector.model.1.num_batches_tracked', 'online_network.projector.model.3.weight', 'online_network.projector.model.3.bias', 'online_network.predictor.model.0.weight', 'online_network.predictor.model.1.weight', 'online_network.predictor.model.1.bias', 'online_network.predictor.model.1.running_mean', 'online_network.predictor.model.1.running_var', 'online_network.predictor.model.1.num_batches_tracked', 'online_network.predictor.model.3.weight', 'online_network.predictor.model.3.bias', 'target_network.encoder.conv1.weight', 'target_network.encoder.bn1.weight', 'target_network.encoder.bn1.bias', 'target_network.encoder.bn1.running_mean', 'target_network.encoder.bn1.running_var', 't

### PyTorch Lightning Class

In [30]:
class Net(pytorch_lightning.LightningModule): # Name: Net **1
    
    # Netz + Loss definieren
    def __init__(self):
        super().__init__()
        
        # Remove the last layer (linear layer) and add a own linear layer 
        # This is needed for the freezing. The Conv part that should be frozen (without the linear layer) needs to be named "backbone". 
        self.net = res_model
        
        if backbone_model == "ResNet":
            # Remove last layer 
            num_filters = self.net.fc.in_features
            layers = list(self.net.children())[:-1]
            self.backbone = torch.nn.Sequential(*layers)
            # Add one linear layer for classification
            self.classifier = torch.nn.Linear(num_filters, num_class)
            del self.net
        
        # Loss
        self.loss_function = loss 
        
        # Metrics
        self.best_acc = torchmetrics.MaxMetric()
    

    def forward(self, x):
        
        x = self.backbone(x).flatten(1)
        x = self.classifier(x)

        return x
    
    def on_train_start(self):
        # by default lightning executes validation step sanity checks before training starts,
        # so we need to make sure val_acc_best doesn't store accuracy from these checks
        self.best_acc.reset()


    def prepare_data(self):
        
        # Creates dataset (path in which all images and labels are located ([vol, class]) + transforms)
        self.train_ds = MedNISTDataset(train_x, train_y, train_transforms)
        self.val_ds = MedNISTDataset(val_x, val_y, val_transforms)
        self.test_ds = MedNISTDataset(test_x, test_y, val_transforms)
        
    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(self.train_ds, batch_size=bs, shuffle=True, num_workers=10)
        print("Train Data Loader | Bs:", bs, "| len",  len(train_loader), "| ges", bs*len(train_loader))
        return train_loader

    def val_dataloader(self):
        val_loader = torch.utils.data.DataLoader(self.val_ds, batch_size=1, num_workers=10)
        print("Val Data Loader", len(val_loader))
        return val_loader
    
    def test_dataloader(self): 
        test_loader = torch.utils.data.DataLoader(self.test_ds, batch_size=1, num_workers=10)
        print("Test Data Loader", len(test_loader))
        return test_loader

    def configure_optimizers(self):
        if optim == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr, weight_decay = WeightDecay)
        return optimizer

    def training_step(self, batch, batch_idx): # batch = PT Dataset
        images, labels= batch 
        
        output = self.forward(images) 
        
        loss = self.loss_function(output, labels) 
        
        # Wandb Logs
        self.log("train/loss", loss.item())

        return {"loss": loss}
    
    def training_epoch_end(self, outputs): # outputs = Sampels in the train dataset  x {"loss": loss, "log": tensorboard_logs}
        train_loss, num_items = 0, 0
        
        for output in outputs: # Loops through all smaples in the train dataset

            train_loss += output["loss"].sum().item() # Adds all losses 
            num_items += 1
            
        mean_train_loss = torch.tensor(train_loss / num_items) # mean loss
        
        # Wandb Logs
        self.log("train/mean_loss", mean_train_loss,)
    

    def validation_step(self, batch, batch_idx): 
        images, labels= batch 
        
        
        outputs = self.forward(images)
        
        klasse = torch.argmax(outputs, dim=1)
        
        loss = self.loss_function(outputs, labels)
        
        return {"val_loss": loss, "targets": labels, "preds": klasse}
    

    def validation_epoch_end(self, outputs): # outputs = Samples in val dataset x {"val_loss": loss, "targets": labels, "preds": klasse}
        val_loss, num_items, true, false = 0, 0, 0, 0 
        
        
        for output in outputs: # Loops through all smaples in the val dataset 
            val_loss += output["val_loss"].sum().item() # Adds all losses  
            num_items += 1
            
            # How many classified correctly
            if output["preds"] == output["targets"]: # if index with highest probability == correct class -> correctly classified
                true += 1
            else:
                false += 1
            
        mean_val_loss = torch.tensor(val_loss / num_items) # mean loss
        
        # Accurancy
        acc = torch.tensor(true / num_items)
        self.best_acc.update(acc)
        best_acc = self.best_acc.compute()
        
        # Wandb Logs 
        self.log("val/loss", mean_val_loss)
        self.log("val/accuracy", acc)
        self.log("val/best_accuracy", best_acc)
        

    
    def test_step(self, batch, batch_idx): 
        images, labels = batch 
        
        outputs = self.forward(images) 
        outputs = torch.nn.functional.softmax(outputs)
        
        klasse = torch.argmax(outputs, dim=1)
        
        loss = self.loss_function(outputs, labels)
        
        return {"test_loss": loss, "output": outputs, "targets": labels, "preds": klasse} 
    
    
    def test_epoch_end(self, outputs): # outputs = Samples in test dataset x  {"test_loss": loss, "output": outputs, "lable": labels, "klasse": klasse}
        
        test_loss, true, false, num_items = 0, 0, 0, 0
        
        output_list = np.array([])
        lable_list = np.array([])
        output_prob = np.array([])
        
        i = 0
        for output in outputs: # Loops through all smaples in the val dataset

            test_loss += output["test_loss"] 
            num_items += 1
            
            output_hold = output["preds"].cpu().detach()
            lable_hold = output["targets"].cpu().detach()
            output_prob_hold = output["output"].cpu().detach()
            
            output_list = np.append(output_list, output_hold) 
            lable_list = np.append(lable_list, lable_hold) 
            output_prob = np.append(output_prob, output_prob_hold[...,1]) # Output from network: [0.4524, 0.5476] -> take only the second value = Propablility score class 0 and 1
            

            if output["preds"] == output["targets"]: 
                true += 1
            else:
                false += 1
            i+=1
            
        output_list = output_list.astype(int)
        lable_list = lable_list.astype(int)
        output_prob = output_prob
        
        
        # __________ DOKU ____________________________
        
        
        # Number correctly classified
        print("\n" + "True", true) 
        print("False", false)
        
        # Accurancy
        acc = torch.tensor(true / num_items) 
        print("\n" + "acc", acc)
        
        # AUC
        auc = roc_auc_score(lable_list, output_prob)
        print("\n" + "AUC",auc)
        
        # F1
        f1 = f1_score(lable_list, output_list)
        print("\n" + "F1",f1)
        
        # precission
        precision = precision_score(lable_list, output_list)
        print("\n" + "precision", precision)
        
        # Mean Loss
        mean_test_loss = torch.tensor(test_loss / num_items) # mittlerer Loss
        print("\n" + "mean_loss", mean_test_loss)
        
        # Confusion Matrix 
        cm = confusion_matrix(lable_list, output_list)
        print("\n", cm)
        
        # Report: Precision, Recall, F-Score
        print("\n" +"Classification Report: ")
        score01 = classification_report(lable_list, output_list)
        print("\n", score01)

        # Wandb Logs
        self.log("test/Loss", mean_test_loss)
        self.log("test/Accuracy", acc)
        self.log("test/AUC", auc)
        self.log("test/F1", f1)
        self.log("test/Precision", precision)

### Train, Test, Save

In [32]:
# initialise the LightningModule
net = Net()

# Creates a path to save the checkpoints and WandB infos: 
checkpoint_dir = os.path.join(root_dir, Project, Run) 
try:
    os.makedirs(checkpoint_dir) 
except OSError:
    print ("Path %s exsists" % checkpoint_dir)
else:
    print ("Successfully created path %s" % checkpoint_dir)

    
# weights and biases
wandb_logger = WandbLogger(project=Project, name=Run, tags = wandb_tag, save_dir=checkpoint_dir)


## Callbacks  

# Saves the Checkpoints 
checkpoint_callback = callbacks.ModelCheckpoint( # opens a new folder each time (logs-"test1-{epoch}-{val_loss:.2f}") Stores the statu
    dirpath = checkpoint_dir, # Saves path
    monitor = "val/accuracy", # What I want to write in filename I have to monitor here 
    mode="max", 
    filename = "{epoch}",
    save_last = True, # Saves the status of the last epoch of training (Name: last.ckpt)
    save_top_k = 2, # Saves the status of the k=2 best epochs [best val accuracy] of the training  (Name: epoch=*-val_loss=*.ckpt)
)
# Set up to which epoch you want to freeze 
# What you want to freeze must be called "backbone"
finetuning = callbacks.BackboneFinetuning(
    unfreeze_backbone_at_epoch=unfreeze, 
    )

# Loggt Lr in Wandb
lr_monitor = callbacks.LearningRateMonitor(logging_interval=None, log_momentum=True)


# All Callbacks together  (from here and from Hydra [wbcall von wandb_callbacks.py, von hydra template])
if first_frozen == True:
    callback_summary = [checkpoint_callback, finetuning, lr_monitor, wbcall.LogConfusionMatrix(), wbcall.LogF1PrecRecHeatmap()] #wbcall.LogF1AUCTest()
else:
    callback_summary = [checkpoint_callback, lr_monitor, wbcall.LogConfusionMatrix(), wbcall.LogF1PrecRecHeatmap()] #wbcall.LogF1AUCTest()


# Netz printen
#print(summary(Net(), torch.zeros((1, 3, 224, 224)), show_input=True))
#print(summary(Net(), torch.zeros((1, 3, 224, 224)), show_input=False))
#print(summary(Net(), torch.zeros((1, 3, 224, 224)), show_input=False, show_hierarchical=True))


# initialise Lightning's trainer.
trainer = pytorch_lightning.Trainer(
    gpus=1,
    max_epochs=Epochen, # Epochs
    logger=wandb_logger, # weight & biasis 
    log_every_n_steps=9, # How often does it log (every 9 steps) 
    callbacks=callback_summary,
    num_sanity_val_steps=1, # Does a run with the Val dataset to check if everything fits
)
trainer.logger.log_hyperparams(info) # So that my information is logged in wandb

# train + val
trainer.fit(net)

# test
ckpt_path = trainer.checkpoint_callback.best_model_path
print(ckpt_path)
trainer.test(net, ckpt_path=ckpt_path)

Successfully created path /home/wolfda/Data/Spark/Downstream/Results/Brain/SparK_1


wandb: Currently logged in as: wolfda95. Use `wandb login --relogin` to force relogin


/home/wolfda/anaconda3/envs/down_xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


2023-11-23 13:27:02,623 - GPU available: True (cuda), used: True
2023-11-23 13:27:02,624 - TPU available: False, using: 0 TPU cores
2023-11-23 13:27:02,624 - IPU available: False, using: 0 IPUs
2023-11-23 13:27:02,625 - HPU available: False, using: 0 HPUs


/home/wolfda/anaconda3/envs/down_xai/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/wolfda/Data/Spark/Downstream/Results/Brain/SparK_1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


...

/tmp/ipykernel_237804/1510233035.py:205: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean_test_loss = torch.tensor(test_loss / num_items) # mittlerer Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/AUC          │    0.6523809523809524     │
│       test/Accuracy       │    0.5517241358757019     │
│          test/F1          │    0.23529411764705882    │
│         test/Loss         │    0.6906571388244629     │
│      test/Precision       │    0.6666666666666666     │
└───────────────────────────┴───────────────────────────┘

[{'test/Loss': 0.6906571388244629,
  'test/Accuracy': 0.5517241358757019,
  'test/AUC': 0.6523809523809524,
  'test/F1': 0.23529411764705882,
  'test/Precision': 0.6666666666666666}]

<Figure size 1400x800 with 0 Axes>

<Figure size 1400x300 with 0 Axes>

In [17]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}